In [ ]:
import numpy as np
import pandas as pd

In [ ]:
path = 'my_naics_chg/naics_'
codes = ['11', '21', '51', '52', '54']

for code in codes:
    df = pd.read_csv(path + code + '.csv')

In [ ]:
df.columns

In [ ]:
# use Kurt's 2020 BEA classifications to calculate 
temp_list = []

for col in df.columns[8:12]:
    s = df.groupby('type_kurt20')[col].sum()
    temp_list.append(s)

mynaics = pd.concat(temp_list, axis=1)
mynaics

In [ ]:
# load Kurt's NAICS data

num = '51'
kw = pd.read_csv('NAICS/NAICS_' + num + '_Employment_Counties.csv')

temp_list = []

kw['TYPE_y'].fillna('RURAL', inplace=True)

for col in kw.columns[20:24]:
    s = kw.groupby('TYPE_y')[col].sum()
    temp_list.append(s)

kwnaics = pd.concat(temp_list, axis=1)
kwnaics

In [ ]:
# compare mine and Kurt's data using same classification scheme
(mynaics - kwnaics.values)# / kwnaics.values 

In [ ]:
# merge to see from where discrepancy arises
merged = df.merge(kw, how='left', left_on='fips', right_on='GEOID')
merged.head()

In [ ]:
# using same metro/micro classification scheme
# difference must arise from either different numbers or missing counties
cols = merged.columns[:8].tolist() + ['TYPE_y'] + merged.columns[8:12].tolist() + merged.columns[36:40].tolist()
diff = merged[merged['type_kurt20'] != merged['TYPE_y'].str[:5].str.title()][cols]

# all nonmatching typologies do not appear in Kurt's data
diff.dropna(subset='TYPE_y').shape

In [ ]:
# difference between mine and Kurt's rural 1990 #s
distance = 1044

# calculates the jobs from all the rural counties NOT in Kurt's dataset
missing = diff[diff['type_kurt20'] == 'Rural']['annual_avg_emplvl_90'].sum()

# calculates all the 1s created for calculating percentages
ones = (merged[merged['TYPE_y'] == 'RURAL']['1990 EMPLOYMENT'] == 1).astype(int).sum()

# the difference should be equal to all the missing jobs counted in mine, minues the ones added to Kurt's 
distance == missing - ones

In [ ]:
# the same should hold for all years:
distances = [1044, 1216, 1545, 1093]
check_cols = diff.columns[9:]

for x in range(0, 4):
    missing = diff[diff['type_kurt20'] == 'Rural'][check_cols[x]].sum()
    ones = (merged[merged['TYPE_y'] == 'RURAL'][check_cols[x+4]] == 1).astype(int).sum()
    print(check_cols[x], missing - ones, distances[x], missing, ones, missing - ones == distances[x])

In [ ]:
# explore why the 2020 variable doesn't match up

# create rural dataframe of just the counties where the 2020 numbers don't align & drop nulls
rural = merged[merged['type_kurt20'] == 'Rural']
ruralunmatched20 = rural[rural['annual_avg_emplvl_20'] != rural['2020 EMPLOYMENT']].dropna(subset='2020 EMPLOYMENT')

# calculate and store the difference between Kurt and my calculations and add it to the difference
z = (ruralunmatched20['annual_avg_emplvl_20'] - ruralunmatched20['2020 EMPLOYMENT']).sum()
print(check_cols[x], missing, ones, z, missing - ones + z, distances[x], missing - ones + z == distances[x])

In [ ]:
# most of the counties differ by 1s, which might be due to coding errors 
(ruralunmatched20['annual_avg_emplvl_20'] - ruralunmatched20['2020 EMPLOYMENT']).value_counts()

In [ ]:
# but 7 counties have employment differences >1
ruralunmatched20['diff'] = ruralunmatched20['annual_avg_emplvl_20'] - ruralunmatched20['2020 EMPLOYMENT']
ruralunmatched20[cols + ['diff']][abs(ruralunmatched20['diff']) > 1]

In [ ]:
# what would have led these numbers to be this far apart?
# return to BLS3 to check on this?
ruralunmatched20[cols + ['diff']][abs(ruralunmatched20['diff']) > 1]['fips'].tolist()